# RuBERT для извлечения именованных сущностей (Бейзлайн для соревнования RuNNE)

## Использование ноутбука

Данный ноутбук полностью инициализирует бейзлайн модель, загружает данные (представленные в формате *brat*), проводит на них обучение и тестирование. \
Скрипт оценивания `score.py` полностью соотвествует базовому в `../scoring_program` за исключением параметризации выбора файла для загрузки. 

Перед запуском ноутбука требуется:
1. Создать директорию `<IN_PATH>` - полное дерево по формату скрипта оценивания `score.py` (`ref`, `res`, а также файл со всеми сущностями `ners.txt`, см. `score.py`);
2. Скачать словарь - файл `vocab.txt` [отсюда](https://huggingface.co/DeepPavlov/rubert-base-cased/blob/main/vocab.txt);
3. Помимо самих данных, скачать оба файла `train.jsonl` и `dev.jsonl` из `../public_data` для автоматической проверки;
4. Создать директорию для сохранения логов и чекпойнтов для обучения. 

Бейзлайн качество в лидерборде было получено при использовании этого ноутбука. Для проверки требуется получившийся после тестирования файл `<ВАШ IN_PATH>/res/dev.jsonl` переименовать в `test.jsonl` и загрузить в систему в виде zip-архива.

## Необходимые пакеты

- PyTorch
- PyTorch Lightning 
- HuggingFace Transformers
- HuggingFace Tokenizers
- NLTK (+punkt)
- tqdm

## Загрузка библиотек и файлов

In [ ]:
import pytorch_lightning as pl
import torch
import json
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer

from transformers import BertForTokenClassification, AdamW, get_linear_schedule_with_warmup, logging
from tokenizers import BertWordPieceTokenizer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

from baseline import BaselineRuBERT
from iobes_flat_dataset import IOBESFlatRuNNEDataset, collate_to_max_length
from score import Evaluator

## Параметры для обучения и/или тестирования модели

Эти параметры можно (и нужно) менять для контролирования обучения, и других различных целей.

In [ ]:
VOCAB_PATH = "./for_gits/vocab.txt"
NERS_PATH = "./for_gits/eval/ref/ners.txt"
IN_PATH = "./for_gits/eval"
OUT_PATH = "./for_gits/eval"

TRAIN_PATH = "./RuNNE/data/train"
DEV_PATH = "./RuNNE/data/dev"
# TEST_PATH = "./data/test"

TRAIN_IDS_PATH = "./for_gits/train.jsonl"
DEV_IDS_PATH = "./for_gits/dev.jsonl"
# TEST_IDS_PATH = "./data/test.jsonl"

CKPT_PATH = "./for_gits/checkpoints"
CKPT_FILE = "./RuNNE/ckpt/epoch=206-step=37466.ckpt"

MAX_LEN = 128
BATCH_SIZE = 8
NUM_WORKERS = 8
MAX_EPOCHS = 1
LR = 1e-4
WEIGHT_DECAY = 0.02

## Загрузка данных

In [ ]:
logging.set_verbosity_error()

In [ ]:
bertwptokenizer = BertWordPieceTokenizer(VOCAB_PATH, lowercase=False)

In [ ]:
train_dataset = IOBESFlatRuNNEDataset( dataset_name = "train",
                                       dataset_path = TRAIN_PATH, 
                                       ners_path = NERS_PATH, 
                                       format_path = TRAIN_IDS_PATH,
                                       in_path = IN_PATH,
                                       tokenizer = bertwptokenizer, 
                                       max_length = MAX_LEN )
train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    collate_fn = collate_to_max_length
)

In [ ]:
dev_dataset   = IOBESFlatRuNNEDataset( dataset_name = "dev",
                                       dataset_path = DEV_PATH,   
                                       ners_path = NERS_PATH, 
                                       format_path = DEV_IDS_PATH,
                                       in_path = IN_PATH,
                                       tokenizer = bertwptokenizer, 
                                       max_length = MAX_LEN )
dev_dataloader = DataLoader(
    dataset = dev_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    collate_fn = collate_to_max_length
)

## Инициализация модели и трейнера для обучения

In [ ]:
model = BaselineRuBERT (
    in_path = IN_PATH,
    out_path = OUT_PATH,
    tag_to_id = train_dataset.tag_to_id,
    total_steps = (len(train_dataset) // BATCH_SIZE) * MAX_EPOCHS,
    lr = LR,
    weight_decay = WEIGHT_DECAY
)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    # Директория, куда будут сохраняться чекпойнты и логи (по умолчанию корневая папка проекта)
    dirpath = CKPT_PATH,
    save_top_k = 1,
    verbose = True,
    monitor = 'macro_f1',
    mode = "max", # Сохраняем самые максимальные по метрике модели
)

In [ ]:
trainer = Trainer(
    gpus = -1,
    callbacks = [checkpoint_callback],
    num_sanity_val_steps = -1,
    max_epochs = 1
)

## Обучение

In [ ]:
trainer.fit(model, train_dataloader, dev_dataloader) # Запуск процесса обучения и валидации, с мониторингом

## Инициализация модели из чекпойнта и трейнера для тестирования

In [ ]:
ckpt_model = BaselineRuBERT.load_from_checkpoint(
    CKPT_FILE, 
    in_path = IN_PATH,
    out_path = OUT_PATH,
    tag_to_id = dev_dataset.tag_to_id,
    total_steps = 0, 
    lr = LR,
    weight_decay = WEIGHT_DECAY
)

In [ ]:
val_trainer = Trainer(
    gpus = -1
)

## Тестирование

In [ ]:
val_trainer.validate(ckpt_model, dataloaders = dev_dataloader)